### Here, I see if I can pick up the Berry phase for SNAP gate.

##### In V4.1, I remove the $e^{i \omega_c t}$ factor. So I use:

$$ \hat{H}_1 = \epsilon(t) \hat{a}^\dag + H.c. $$

$$\alpha = i \int \epsilon(t) dt$$

In [101]:
import numpy as np
from scipy.linalg import expm
import scipy
import matplotlib.pyplot as plt
import time

In [102]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)

In [103]:
cavity_dim = 8
omega_q = 21
omega_c = 20
Delta = omega_q - omega_c
chi = 1 / Delta

T = 1
dt = 0.01
t_axis = np.arange(0, T, dt)[0:]

In [104]:
sigmaI = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)
sigmaX = np.array(
    [
        [0, 1],
        [1, 0],
    ]
)
# sigmaZ = np.array(
#     [
#         [-1, 0],
#         [0, 1],
#     ]
# )
sigmaBin = np.array(
    [
        [0, 0],
        [0, 1],
    ]
)

# Annihilation operator
a = np.zeros((cavity_dim, cavity_dim))
n_indices = np.arange(1, cavity_dim)
a[n_indices - 1, n_indices] = np.sqrt(n_indices)
a_dag = a.conjugate().transpose()
# print(a)
# print(a_dag)

def Fock(n):
    vec = [[0] for _ in range(cavity_dim)]
    vec[n] = [1]
    return np.array(vec)
# print(Fock(1))

In [105]:
drive_qubit = np.array([0 for t in t_axis])
drive_cavity = np.array([18.75/T for t in t_axis])

def H(i):
    # from the SNAP paper, alpha = 1j * integrate(drive_cavity * dt)
    return (
        omega_q * np.kron(sigmaBin, np.identity(cavity_dim))
      + (omega_c - chi*np.kron(sigmaBin, np.identity(cavity_dim))) * (np.kron(sigmaI, a_dag) @ np.kron(sigmaI, a))
      + drive_cavity[i] * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.kron(sigmaI, a)
      + drive_qubit[i] * np.kron(np.array([[0,0],[1,0]]), np.identity(cavity_dim)) + drive_qubit[i].conjugate() * np.kron(np.array([[0,1],[0,0]]), np.identity(cavity_dim))
    )

def D(alpha):
    return expm(alpha * np.kron(sigmaI, a_dag) - np.conjugate(alpha) * np.kron(sigmaI, a))
# def D(alpha):
#     return expm(alpha * a_dag - np.conjugate(alpha) * a)

def CoheState(alpha):
    ket = np.zeros((cavity_dim, 1), dtype=np.complex128)
    ket[0] = 1. + 0j
    for n in range(1, cavity_dim):
        ket[n] = ket[n-1] * alpha / np.sqrt(n)
    return ket / np.linalg.norm(ket)
    # return ket * expm(-np.abs(alpha)**2 / 2)
    

In [106]:
1j * sum(drive_cavity * dt)

18.75j

In [107]:
# H(len(t_axis)-1)
# eigenVals, eigenVecs = np.linalg.eigh(H(0))

In [108]:
Unitary = np.identity(len(H(0)))
for i in range(len(t_axis)):
    Unitary = expm(-1j * H(i) * dt) @ Unitary
print(Unitary)

[[-0.24 -0.556j -0.209+0.374j  0.421-0.233j -0.247-0.115j  0.014+0.254j
   0.087-0.211j -0.087+0.124j  0.046-0.052j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [-0.209+0.374j  0.133-0.487j  0.176-0.168j -0.031-0.264j -0.238+0.382j
   0.283-0.252j -0.22 +0.141j  0.116-0.063j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.421-0.233j  0.176-0.168j -0.07 -0.772j  0.007+0.018j -0.144-0.087j
   0.037+0.237j -0.058-0.11j   0.083+0.017j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [-0.247-0.115j -0.031-0.264j  0.007+0.018j -0.206-0.645j  0.073-0.092j
  -0.43 +0.376j  0.215-0.026j  0.071-0.066j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.014+0.254j -0.238+0.382j -0.144-0.087j  0.073-0.092j -0.524-0.199j
  -0.146+0.06j  -0.0

In [109]:
# Unitary @ np.kron( np.array([[1], [0]]), Fock(0))

np.abs(Unitary @ np.kron( np.array([[1], [0]]), Fock(0)))**2

array([[0.366],
       [0.183],
       [0.232],
       [0.074],
       [0.065],
       [0.052],
       [0.023],
       [0.005],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])

In [110]:
# D(1j) @ np.kron( np.array([[1], [0]]), Fock(0))

np.abs(D(1j) @ np.kron( np.array([[1], [0]]), Fock(0)))**2

array([[0.368],
       [0.368],
       [0.184],
       [0.061],
       [0.015],
       [0.003],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])

In [111]:
# np.kron( np.array([[1], [0]]), CoheState(1j))

np.abs(np.kron( np.array([[1], [0]]), CoheState(1j)))**2

array([[0.368],
       [0.368],
       [0.184],
       [0.061],
       [0.015],
       [0.003],
       [0.001],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])

In [112]:
def H(i):
    # from the SNAP paper, alpha = 1j * integrate(drive_cavity * dt)
    return (
        omega_q * np.kron(sigmaBin, np.identity(cavity_dim))
      + (omega_c - chi*np.kron(sigmaBin, np.identity(cavity_dim))) * (np.kron(sigmaI, a_dag) @ np.kron(sigmaI, a))
      + drive_cavity[i] * np.exp(1j * omega_c * i*dt) * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.exp(-1j * omega_c * i*dt) * np.kron(sigmaI, a)
      + drive_qubit[i] * np.exp(1j * omega_q * i*dt) * np.kron(np.array([[0,0],[1,0]]), np.identity(cavity_dim)) + drive_qubit[i].conjugate() * np.exp(-1j * omega_q * i*dt) * np.kron(np.array([[0,1],[0,0]]), np.identity(cavity_dim))
    )

H(len(t_axis)-1)

Unitary = np.identity(len(H(0)))
for i in range(len(t_axis)):
    Unitary = expm(-1j * H(i) * dt) @ Unitary
print(Unitary)

[[-0.466+0.51j   0.566-0.181j -0.353-0.076j  0.134+0.118j -0.023-0.074j
  -0.005+0.032j  0.005-0.011j -0.002+0.003j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.477+0.355j  0.073+0.164j  0.065-0.528j -0.263+0.379j  0.278-0.109j
  -0.159+0.002j  0.064+0.008j -0.02 -0.002j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.187-0.309j  0.467-0.254j -0.122+0.021j -0.282-0.12j   0.206+0.505j
  -0.077-0.379j  0.051+0.157j -0.032-0.045j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [-0.162-0.075j -0.273-0.371j -0.067-0.299j -0.29 +0.399j -0.451+0.095j
   0.38 -0.128j -0.038+0.159j -0.055-0.125j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [-0.027+0.073j -0.235+0.184j -0.545+0.032j -0.339-0.312j  0.023-0.137j
  -0.168+0.116j  0.0

In [113]:
# Unitary @ np.kron( np.array([[1], [0]]), Fock(0))

np.abs(Unitary @ np.kron( np.array([[1], [0]]), Fock(0)))**2

array([[0.477],
       [0.353],
       [0.131],
       [0.032],
       [0.006],
       [0.001],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])

In [114]:
# D(1j) @ np.kron( np.array([[1], [0]]), Fock(0))

np.abs(D(1j) @ np.kron( np.array([[1], [0]]), Fock(0)))**2

array([[0.368],
       [0.368],
       [0.184],
       [0.061],
       [0.015],
       [0.003],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])

In [115]:
# np.kron( np.array([[1], [0]]), CoheState(1j))

np.abs(np.kron( np.array([[1], [0]]), CoheState(1j)))**2

array([[0.368],
       [0.368],
       [0.184],
       [0.061],
       [0.015],
       [0.003],
       [0.001],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])